Extract financial data and historical stock prices from yahoo
For web info in json format

In [73]:
import re
import json
from io import StringIO
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd



In [3]:
url_stats = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile = 'https://finance.yahoo.com/quote/{}/profile?p={}'
url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'

In [8]:
stock = 'F'

yahoo finance use cookies to track incoming requests.
We need to hack the headers on requests to get this to run


In [28]:
## hack the request to fool yahoo finance
web_header={'Connection': 'keep-alive',
            'Expires': '-1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
           }

In [30]:
### response = requests.get(url_financials.format(stock, stock))
response = requests.get(url_financials.format(stock, stock), headers=web_header)


In [33]:
response.text


'<!DOCTYPE html><html data-color-theme="light" id="atomic" class="NoJs chrome desktop" lang="en-US"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark(\'PageStart\');</script><meta charSet="utf-8"/><title>Ford Motor Company (F) Income Statement - Yahoo Finance</title><meta name="keywords" content="income statement,gross profit,revenue,operating expenses,operating income,net income,earning,earning per share"/><meta http-equiv="x-dns-prefetch-control" content="on"/><meta property="twitter:dnt" content="on"/><meta property="fb:app_id" content="458584288257241"/><meta name="theme-color" content="#400090"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="description" lang="en-US" content="Get the detailed quarterly/annual income statement for Ford Motor Company (F). Find out the revenue, expenses and profit or loss over the last fiscal year."/><meta name="oath:guce:consent-host" content="guce.

In [34]:
soup = BeautifulSoup(response.text, 'html.parser')


search for /* -- Data -- */


In [35]:
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]


In [37]:
script_data[:500]

'\n(function (root) {\n/* -- Data -- */\nroot.App || (root.App = {});\nroot.App.now = 1664547030127;\nroot.App.main = {"context":{"dispatcher":{"stores":{"PageStore":{"currentPageName":"quote","currentEvent":{"eventName":"NEW_PAGE_SUCCESS"},"currentRenderTargetId":"default","pagesConfigRaw":{"base":{"quote":{"layout":{"bundleName":"yahoodotcom-layout.TwoColumnLayout","name":"TwoColumnLayout","config":{"enableHeaderCollapse":true,"additionalBodyWrapperClasses":"Bgc($layoutBgColor)!","contentWrapperClas'

find the boundary of the json data - from context to -12 characters from end

In [38]:
start = script_data.find("context")-2


In [39]:
json_data = json.loads(script_data[start:-12])

In [40]:
json_data['context'].keys()


dict_keys(['dispatcher', 'options', 'plugins'])

use dataFrame to explore the data items

In [95]:
j_df = pd.DataFrame(json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory'])
j_df

,researchDevelopment,effectOfAccountingCharges,incomeBeforeTax,minorityInterest,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,...,otherItems,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,maxAge,discontinuedOperations,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares
0,{},{},"{'raw': 17780000000, 'fmt': '17.78B', 'longFmt...","{'raw': 103000000, 'fmt': '103M', 'longFmt': '...","{'raw': 17937000000, 'fmt': '17.94B', 'longFmt...","{'raw': 5918000000, 'fmt': '5.92B', 'longFmt':...","{'raw': 16507000000, 'fmt': '16.51B', 'longFmt...","{'raw': 10589000000, 'fmt': '10.59B', 'longFmt...","{'raw': 1640908800, 'fmt': '2021-12-31'}","{'raw': 10589000000, 'fmt': '10.59B', 'longFmt...",...,{},"{'raw': -130000000, 'fmt': '-130M', 'longFmt':...","{'raw': 136341000000, 'fmt': '136.34B', 'longF...","{'raw': 125752000000, 'fmt': '125.75B', 'longF...","{'raw': 119834000000, 'fmt': '119.83B', 'longF...","{'raw': 7191000000, 'fmt': '7.19B', 'longFmt':...",1,{},"{'raw': 17910000000, 'fmt': '17.91B', 'longFmt...","{'raw': 17937000000, 'fmt': '17.94B', 'longFmt..."
1,{},{},"{'raw': -1116000000, 'fmt': '-1.12B', 'longFmt...","{'raw': 121000000, 'fmt': '121M', 'longFmt': '...","{'raw': -1279000000, 'fmt': '-1.28B', 'longFmt...","{'raw': 10124000000, 'fmt': '10.12B', 'longFmt...","{'raw': 6123000000, 'fmt': '6.12B', 'longFmt':...","{'raw': -4001000000, 'fmt': '-4B', 'longFmt': ...","{'raw': 1609372800, 'fmt': '2020-12-31'}","{'raw': -4001000000, 'fmt': '-4B', 'longFmt': ...",...,{},"{'raw': 160000000, 'fmt': '160M', 'longFmt': '...","{'raw': 127144000000, 'fmt': '127.14B', 'longF...","{'raw': 131145000000, 'fmt': '131.15B', 'longF...","{'raw': 121021000000, 'fmt': '121.02B', 'longF...","{'raw': 2885000000, 'fmt': '2.88B', 'longFmt':...",1,{},"{'raw': -1276000000, 'fmt': '-1.28B', 'longFmt...","{'raw': -1279000000, 'fmt': '-1.28B', 'longFmt..."
2,{},{},"{'raw': -640000000, 'fmt': '-640M', 'longFmt':...","{'raw': 45000000, 'fmt': '45M', 'longFmt': '45...","{'raw': 47000000, 'fmt': '47M', 'longFmt': '47...","{'raw': 12763000000, 'fmt': '12.76B', 'longFmt...","{'raw': 12876000000, 'fmt': '12.88B', 'longFmt...","{'raw': 113000000, 'fmt': '113M', 'longFmt': '...","{'raw': 1577750400, 'fmt': '2019-12-31'}","{'raw': 113000000, 'fmt': '113M', 'longFmt': '...",...,{},"{'raw': -724000000, 'fmt': '-724M', 'longFmt':...","{'raw': 155900000000, 'fmt': '155.9B', 'longFm...","{'raw': 155787000000, 'fmt': '155.79B', 'longF...","{'raw': 143024000000, 'fmt': '143.02B', 'longF...","{'raw': -753000000, 'fmt': '-753M', 'longFmt':...",1,{},"{'raw': 84000000, 'fmt': '84M', 'longFmt': '84...","{'raw': 47000000, 'fmt': '47M', 'longFmt': '47..."
3,{},{},"{'raw': 4345000000, 'fmt': '4.34B', 'longFmt':...","{'raw': 134000000, 'fmt': '134M', 'longFmt': '...","{'raw': 3677000000, 'fmt': '3.68B', 'longFmt':...","{'raw': 9781000000, 'fmt': '9.78B', 'longFmt':...","{'raw': 14835000000, 'fmt': '14.84B', 'longFmt...","{'raw': 5054000000, 'fmt': '5.05B', 'longFmt':...","{'raw': 1546214400, 'fmt': '2018-12-31'}","{'raw': 5054000000, 'fmt': '5.05B', 'longFmt':...",...,{},"{'raw': 650000000, 'fmt': '650M', 'longFmt': '...","{'raw': 160338000000, 'fmt': '160.34B', 'longF...","{'raw': 155284000000, 'fmt': '155.28B', 'longF...","{'raw': 145503000000, 'fmt': '145.5B', 'longFm...","{'raw': -709000000, 'fmt': '-709M', 'longFmt':...",1,{},"{'raw': 3695000000, 'fmt': '3.69B', 'longFmt':...","{'raw': 3677000000, 'fmt': '3.68B', 'longFmt':..."


In [43]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews'])

In [48]:
annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']


In [55]:
### extract info from income state json
annual_is_stmts = []
for s in annual_is:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_is_stmts.append(statement)

In [58]:
annual_is_stmts[0]


{'incomeBeforeTax': 17780000000,
 'minorityInterest': 103000000,
 'netIncome': 17937000000,
 'sellingGeneralAdministrative': 5918000000,
 'grossProfit': 16507000000,
 'ebit': 10589000000,
 'endDate': 1640908800,
 'operatingIncome': 10589000000,
 'interestExpense': -1803000000,
 'incomeTaxExpense': -130000000,
 'totalRevenue': 136341000000,
 'totalOperatingExpenses': 125752000000,
 'costOfRevenue': 119834000000,
 'totalOtherIncomeExpenseNet': 7191000000,
 'netIncomeFromContinuingOps': 17910000000,
 'netIncomeApplicableToCommonShares': 17937000000}

how about reading to pandas with json reader

In [82]:
#### j_df = pd.read_json(script_data[start:-12])   ### this dont work..

In [89]:
j_df = pd.DataFrame(annual_is_stmts)
j_df

,incomeBeforeTax,minorityInterest,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares
0,17780000000,103000000,17937000000,5918000000,16507000000,10589000000,1640908800,10589000000,-1803000000,-130000000,136341000000,125752000000,119834000000,7191000000,17910000000,17937000000
1,-1116000000,121000000,-1279000000,10124000000,6123000000,-4001000000,1609372800,-4001000000,-1651000000,160000000,127144000000,131145000000,121021000000,2885000000,-1276000000,-1279000000
2,-640000000,45000000,47000000,12763000000,12876000000,113000000,1577750400,113000000,-1049000000,-724000000,155900000000,155787000000,143024000000,-753000000,84000000,47000000
3,4345000000,134000000,3677000000,9781000000,14835000000,5054000000,1546214400,5054000000,-1228000000,650000000,160338000000,155284000000,145503000000,-709000000,3695000000,3677000000


historical data

In [61]:
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/F?period1=1568483641&period2=1600106041&interval=1d&events=history'

In [62]:
response = requests.get(stock_url, headers=web_header)

In [64]:
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
params = { 'period1' : '1568483641',
          'period2': '1600106041', 
          'interval' : '1d',
          'events': 'history'
         }

In [65]:

params = { 'range': '5y',
          'interval' : '1d',
          'events': 'history'
         }

In [66]:
response = requests.get(stock_url.format(stock), headers=web_header, params=params)

In [71]:
file = StringIO(response.text)
import csv
reader = csv.reader(file)
data = list(reader)
for row in data[:5]:
    print(row)


['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
['2017-10-02', '12.000000', '12.090000', '11.970000', '12.090000', '10.001823', '36020000']
['2017-10-03', '12.320000', '12.430000', '12.220000', '12.340000', '10.208643', '65641700']
['2017-10-04', '12.330000', '12.460000', '12.280000', '12.300000', '10.175551', '52909800']
['2017-10-05', '12.340000', '12.360000', '12.150000', '12.250000', '10.134189', '48145800']


using Pandas to read csv input

In [74]:
file = StringIO(response.text)
df = pd.read_csv(file)


In [75]:
df


,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-10-02,12.00,12.0900,11.97,12.090,10.001823,36020000
1,2017-10-03,12.32,12.4300,12.22,12.340,10.208643,65641700
2,2017-10-04,12.33,12.4600,12.28,12.300,10.175551,52909800
3,2017-10-05,12.34,12.3600,12.15,12.250,10.134189,48145800
4,2017-10-06,12.20,12.3300,12.19,12.310,10.183825,35091800
...,...,...,...,...,...,...,...
1254,2022-09-26,12.21,12.3800,11.93,11.990,11.990000,89300400
1255,2022-09-27,12.21,12.2200,11.66,11.910,11.910000,87044400
1256,2022-09-28,11.91,12.2400,11.76,12.180,12.180000,72701600
1257,2022-09-29,11.85,11.8800,11.36,11.470,11.470000,84304500
